# @File: run.py
# @Author: Saeed AHmad (email:saeediitb@gmail.com)
# @Date:   2023-06-20 18:08:51
# @Last Modified (Saeed Ahmad) time: 2023-08-13 13:49:54
# This code is for versions python=3.10.8, panda=1.5.1, numpy=1.23.5, hoomd=3.5.0 

In [7]:
import hoomd
from hoomd import md
import numpy as np
import pandas as pd
import virus
from virus.IOframe import *
from virus.system import System
from virus.system import Update_Snap
import sys,os,glob,json
import pickle
ONEDGE=32767

In [8]:
def test_merge(sys):
	#cprint("test merging...",'yellow')
	mergestatus = sys.shell.shell_merge()
	while mergestatus == 1:
		cprint('Force Merge Happen','yellow')
		sys.try_relax()
		mergestatus = sys.shell.shell_merge()
	if mergestatus == -1:
		#cprint("Assemble done from merge")
		sys.try_relax()
		#sys.setup_operation("final1.gsd")
	#else: cprint("no more merge!",'yellow')
	return mergestatus

In [9]:

def run():
	virus_sys = System(param)
	virus_sys.initial()
	snap = hoomd.Snapshot()
	snap.configuration.box = [1.5*param['L'], 1.5*param['L'], 1.5*param['L'], 0, 0, 0]
	snap.particles.types = ['A','B']
	snap.bonds.types = ['out_ver_to_out_ver']
	snap.dihedrals.types = ['capsomer']
	snap.angles.types = ['ds']
	virus_sys.sim.create_state_from_snapshot(snap)
	virus_sys.setup_potential() # whether this should always be called once
	virus_sys.setup_operation() #-----------------------------------
	virus_sys.setup_integrator() #-------------------------
	max_run = virus_sys.try_relax() #---------------------------
	print(snap.particles.types)
	sa
	#frame_out(virus_sys.time, virus_sys.shell)
	n=0
	shell=pickle.loads(pickle.dumps(virus_sys.shell))
	shellE = pickle.loads(pickle.dumps(virus_sys.shellE))
	for n in range(100000):
		virus_sys.time = 0.0
		virus_sys.shell = shell
		virus_sys.shellE = shellE
		energy_frame(virus_sys.time,virus_sys.shellE)#,rate=0.0,mc_type='type')
		while (not virus_sys.shell.test_shell_close()):
			virus_sys.kmc_moves()
			mergestatus = test_merge(virus_sys)
			if len(virus_sys.shell.triangle)>=25:
				break
		frame_out(n,virus_sys.time, virus_sys.shell)
		#frame_out(virus_sys.time, virus_sys.shell,name_file='final_shell.csv')
		if len(virus_sys.shell.triangle)==20:
			print(n, virus_sys.time)
			energy_frame(virus_sys.time,virus_sys.shellE,col_name=False)
			First_passage_Time(n, virus_sys.time)



	cprint(" Final Assemble Done!\nFinal Shell:",'white', attrs = ['reverse'])
	print('Congratulations: Run Completed')

In [1]:
os.chdir(glob.glob("R*")[0])
param = glob.glob('R*.json')
param = param[0]
with open(param) as f: param = json.load(f)
os.chdir("..")
run()

NameError: name 'os' is not defined

In [3]:
import numpy as np

# Create a 6x3 matrix
matrix = np.arange(18).reshape(6, 3)
print(matrix)
# Print the 3x3 sub-matrix starting from the top-left corner
print(matrix[:3, :3])
one_d_array = matrix[:3, :3].sum(axis=0)

print(one_d_array)


[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]
[ 9 12 15]


In [5]:
import numpy as np

def find_perpendicular_point(triangle):
  """
  Finds the coordinate of the point that is 0.5 away and perpendicular
  from the centroid of the given triangle.

  Args:
      triangle: A 3x2 NumPy array representing the triangle coordinates.

  Returns:
      A NumPy array containing the coordinates of the perpendicular point.
  """

  # Calculate the centroid
  centroid = np.mean(triangle, axis=0)

  # Get the direction vector from centroid to any vertex (assuming non-degenerate triangle)
  direction = triangle[0] - centroid

  # Normalize the direction vector
  direction /= np.linalg.norm(direction)

  # Move 0.5 units in the perpendicular direction (rotate by 90 degrees)
  perpendicular_point = centroid + 0.5 * np.array([-direction[1], direction[0]])

  return perpendicular_point

# Example usage
triangle = np.array([[1, 2,1], [3, 1,2], [4, 4,2]])
perpendicular_point = find_perpendicular_point(triangle)

print(perpendicular_point)

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [8]:
import numpy as np

# Given triangle vertices (replace with your actual coordinates)
A = np.array([1, 2,1])
B = np.array([3, 1, 2])
C = np.array([4, 4, 2])

# Calculate centroid
centroid = (A + B + C) / 3

# Calculate normal vector
AB = B - A
AC = C - A
normal_vector = np.cross(AB, AC)

# Normalize the normal vector
normalized_normal = normal_vector / np.linalg.norm(normal_vector)

# Calculate the desired point
distance_from_centroid = 0.5
desired_point = centroid + distance_from_centroid * normalized_normal

print(f"Centroid: {centroid}")
print(f"Desired point: {desired_point}")


Centroid: [2.66666667 2.33333333 1.66666667]
Desired point: [2.4713833  2.39842779 2.12232786]


In [13]:
import numpy as np

# Given triangle vertices (replace with your actual coordinates)
triangle = np.array([[1, 2,1], [3, 1,2], [4, 4,2]])

# Calculate centroid
centroid = np.mean(triangle, axis=0)

# Calculate normal vector
AB = triangle[1] - triangle[0]
AC = triangle[2] - triangle[0]
normal_vector = np.cross(AB, AC)

# Normalize the normal vector
normalized_normal = normal_vector / np.linalg.norm(normal_vector)

# Calculate the desired point
distance_from_centroid = 0.5
desired_point = centroid + distance_from_centroid * normalized_normal

print(f"Centroid: {centroid}")
print(f"Desired point: {desired_point}")

Centroid: [2.66666667 2.33333333 1.66666667]
Desired point: [2.4713833  2.39842779 2.12232786]


In [9]:
import numpy as np

def find_perpendicular_point(triangle):
  """
  Finds the coordinate of the point that is 0.5 away and perpendicular
  from the centroid of the given triangle in 3D space.

  Args:
      triangle: A 3x3 NumPy array representing the triangle coordinates.

  Returns:
      A NumPy array containing the coordinates of the perpendicular point.
  """

  # Calculate the centroid
  centroid = np.mean(triangle, axis=0)

  # Get the direction vector from centroid to any vertex (assuming non-degenerate triangle)
  direction = triangle[0] - centroid

  # Normalize the direction vector
  direction /= np.linalg.norm(direction)

  # Create a perpendicular vector by swapping axes and negating one
  perpendicular_axis = np.array([direction[1], -direction[0], 0])

  # Move 0.5 units in the perpendicular direction
  perpendicular_point = centroid + 0.5 * perpendicular_axis

  return perpendicular_point

# Example usage
triangle = np.array([[1, 2, 1], [3, 1, 2], [4, 4, 2]])
perpendicular_point = find_perpendicular_point(triangle)

print(perpendicular_point)


[2.57537957 2.7897688  1.66666667]
